In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import time

def download(method, url, param=None, data=None, headers = None, timeout=1, maxretries=3):
    if headers == None:
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    try:
        resp = requests.request(method, url, 
                                params=param, data=data, 
                                headers = headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, param, data, headers, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [10]:
html = """
<html>
 <head></head>
 <body>
  <div id="result">
   <p class="row">
    <a class="red">123</a>
    <a class="blue">go to page2</a>
    <div><a>안녕하세요</a></div>
    <a class="blue">go to page2</a>
   </p>
  </div>
 </body>
</html>
"""

In [12]:
dom = BeautifulSoup(html, 'lxml')

In [21]:
dom.select("")

[<a class="red">123</a>, <a class="blue">go to page2</a>]

<설명>
- ID => #어쩌고
- CLASS => .어쩌고
- 태그 => 태그
- 자손(모든 손자손녀) => 공백
- 자식(1촌) => >

<예시>
- selector("div") => 2개 찾음
- selector("div#result") => 1개 찾음
- selector("#result") => 2개 찾음
- selector(.red)
- selector()
- selector(body > div) => 1개 찾음

<주의>
- CLASS 당중 상속
    예) <div class="a b c d e">    =>   div.ab.c.d.e
- 개발자도구
    - 전적으로 믿지 마시오.
    - 소스보기 (ㅇ)

# 구글 실습

In [22]:
url = "https://www.google.com/search"
params = {'q':'수지'}

In [24]:
resp = download('get', url, param=params)

In [25]:
dom = BeautifulSoup(resp.text, 'lxml')

In [40]:
# div의 클래스가 rc인 녀석의 자식 중에서 -> div의 클래스가 r인 녀석 중에서 -> a를 찾아온다.

dom.select("div.rc > div.r > a")[0]

<a href="https://namu.wiki/w/%EC%88%98%EC%A7%80(1994)" ping="/url?sa=t&amp;source=web&amp;rct=j&amp;url=https://namu.wiki/w/%25EC%2588%2598%25EC%25A7%2580(1994)&amp;ved=2ahUKEwjd6fWI767jAhXtw4sBHbUnCQoQFjAAegQIARAB"><h3 class="LC20lb">수지(1994) - 나무위키</h3><br/><div class="TbwUpd"><cite class="iUh30">https://namu.wiki/w/수지(1994)</cite></div></a>

In [34]:
# href라는 attr을 확인합니다.

dom.select("div.rc > div.r > a")[0]['href']

'https://namu.wiki/w/%EC%88%98%EC%A7%80(1994)'

In [35]:
# 태그에 포함되어있는 텍스트만 꺼내준다.
# 태그 안에 자식 태그가 있어도 텍스트만 싹 다 꺼내서 한줄로 붙여준다.

dom.select("div.rc > div.r > a")[0].text

'수지(1994) - 나무위키https://namu.wiki/w/수지(1994)'

In [45]:
[(tag.text, tag['href']) for tag in dom.select("div.rc > div.r > a") if tag.has_attr("href")]

[('수지(1994) - 나무위키https://namu.wiki/w/수지(1994)',
  'https://namu.wiki/w/%EC%88%98%EC%A7%80(1994)'),
 ('수지 - 나무위키https://namu.wiki/w/수지', 'https://namu.wiki/w/%EC%88%98%EC%A7%80'),
 ('수지 (대한민국의 가수) - 위키백과, 우리 모두의 백과사전https://ko.wikipedia.org/wiki/수지_(대한민국의_가수)',
  'https://ko.wikipedia.org/wiki/%EC%88%98%EC%A7%80_(%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EA%B0%80%EC%88%98)'),
 ('수지 공식입장, JYP와 결별→매니지먼트숲 行...누리꾼 "가수 수지는 ...https://www.mk.co.kr/star/hot-issues/view/2019/03/183477/',
  'https://www.mk.co.kr/star/hot-issues/view/2019/03/183477/'),
 ("'국민여동생' 수지, 9년간 몸 담은 JYP 떠난다...전도연·공유와 한솥밥 ...news.chosun.com/site/data/html_dir/2019/03/26/2019032602019.html",
  'http://news.chosun.com/site/data/html_dir/2019/03/26/2019032602019.html'),
 ('#수지 hashtag on Twitterhttps://twitter.com/hashtag/수지',
  'https://twitter.com/hashtag/%EC%88%98%EC%A7%80'),
 ('수지, JYP 9년 만에 떠난다…전속계약 만료 : 방송·연예 : 문화 : 뉴스 ...www.hani.co.kr/arti/culture/entertainment/887453.html',
  'http://www.hani.co.kr/arti/cultu

In [49]:
# 선생님 답

[(_["href"], _.text) for _ in dom.select(".rc > .r a")]

[('https://namu.wiki/w/%EC%88%98%EC%A7%80(1994)',
  '수지(1994) - 나무위키https://namu.wiki/w/수지(1994)'),
 ('https://namu.wiki/w/%EC%88%98%EC%A7%80', '수지 - 나무위키https://namu.wiki/w/수지'),
 ('#', ''),
 ('/search?q=related:https://namu.wiki/w/%25EC%2588%2598%25EC%25A7%2580+%EC%88%98%EC%A7%80&tbo=1&sa=X&ved=2ahUKEwjd6fWI767jAhXtw4sBHbUnCQoQHzALegQIABAE',
  '유사한 페이지'),
 ('https://ko.wikipedia.org/wiki/%EC%88%98%EC%A7%80_(%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EA%B0%80%EC%88%98)',
  '수지 (대한민국의 가수) - 위키백과, 우리 모두의 백과사전https://ko.wikipedia.org/wiki/수지_(대한민국의_가수)'),
 ('#', ''),
 ('https://webcache.googleusercontent.com/search?q=cache:UHT7ZtT8t8MJ:https://ko.wikipedia.org/wiki/%25EC%2588%2598%25EC%25A7%2580_(%25EB%258C%2580%25ED%2595%259C%25EB%25AF%25BC%25EA%25B5%25AD%25EC%259D%2598_%25EA%25B0%2580%25EC%2588%2598)+&cd=13&hl=ko&ct=clnk&gl=kr',
  '저장된\xa0페이지'),
 ('/search?q=related:https://ko.wikipedia.org/wiki/%25EC%2588%2598%25EC%25A7%2580_(%25EB%258C%2580%25ED%2595%259C%25EB%25AF%25BC%25EA%25B5

In [30]:
# 선생님 답 2탄

# ".rc > .r a[href^=http]
# ^= 은 정규식 anchor와 비슷함. ~로 시작하는 것을 찾으란 거임.

[(_["href"], _.text) for _ in dom.select(".rc > .r a[href^=http]")]

[('https://namu.wiki/w/%EC%88%98%EC%A7%80(1994)',
  '수지(1994) - 나무위키https://namu.wiki/w/수지(1994)'),
 ('https://namu.wiki/w/%EC%88%98%EC%A7%80', '수지 - 나무위키https://namu.wiki/w/수지'),
 ('https://ko.wikipedia.org/wiki/%EC%88%98%EC%A7%80_(%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EA%B0%80%EC%88%98)',
  '수지 (대한민국의 가수) - 위키백과, 우리 모두의 백과사전https://ko.wikipedia.org/wiki/수지_(대한민국의_가수)'),
 ('https://webcache.googleusercontent.com/search?q=cache:UHT7ZtT8t8MJ:https://ko.wikipedia.org/wiki/%25EC%2588%2598%25EC%25A7%2580_(%25EB%258C%2580%25ED%2595%259C%25EB%25AF%25BC%25EA%25B5%25AD%25EC%259D%2598_%25EA%25B0%2580%25EC%2588%2598)+&cd=13&hl=ko&ct=clnk&gl=kr',
  '저장된\xa0페이지'),
 ('https://www.mk.co.kr/star/hot-issues/view/2019/03/183477/',
  '수지 공식입장, JYP와 결별→매니지먼트숲 行...누리꾼 "가수 수지는 ...https://www.mk.co.kr/star/hot-issues/view/2019/03/183477/'),
 ('https://webcache.googleusercontent.com/search?q=cache:RIV0oOElK2IJ:https://www.mk.co.kr/star/hot-issues/view/2019/03/183477/+&cd=14&hl=ko&ct=clnk&gl=kr',
  '

# 네이버 실습

In [68]:
# 수지의 연관 검색어를 뽑아오겠다!

url = "https://search.naver.com/search.naver"
params = {"query":"수지", "where":"nexearch"}

In [69]:
resp = download('get', url, param=params)

In [70]:
resp.text

'<!doctype html> <html lang="ko"> <head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=2.0"> <meta property="og:title" content="수지 : 네이버 통합검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="\'수지\'의 네이버 통합검색 결과입니다."> <meta name="description" lang="ko" content="\'수지\'의 네이버 통합검색 결과입니다."> <title>수지 : 네이버 통합검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_140327.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_190711.css"> <link rel="stylesheet" type="text/css" href="https

In [74]:
dom = BeautifulSoup(resp.text, 'lxml')

In [75]:
with open("suzy.html", 'w', encoding='utf-8') as f:
    f.write(dom.prettify())

In [81]:
dom.select("#nx_related_keywords ._related_keyword_ul li > a")

[<a data-area="*q" data-idx="1" href="?where=nexearch&amp;query=%EB%AF%B8%EC%93%B0%EC%97%90%EC%9D%B4+%EC%88%98%EC%A7%80&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0">미쓰에이 수지</a>,
 <a data-area="*q" data-idx="2" href="?where=nexearch&amp;query=%EC%95%84%EC%9D%B4%EC%9C%A0&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0">아이유</a>,
 <a data-area="*q" data-idx="3" href="?where=nexearch&amp;query=%EC%9B%90%EB%8D%94%EB%9E%9C%EB%93%9C&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0">원더랜드</a>,
 <a data-area="*q" data-idx="4" href="?where=nexearch&amp;query=%EC%88%98%EC%A7%80+%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0">수지 나무위키</a>,
 <a data-area="*q" data-idx="5" href="?where=nexearch&amp;query=%EC%88%98%EC%A7%80+%EC%9D%B4%EB%AF%BC%ED%98%B8&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0">수지 이민호</a>,
 <a data-area="*q" data-idx="6" href="?where=nexearch&amp;query=%EB%B0%B0%EA%B0%80%EB%B3%B8%EB%93%9C&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0">배가본드</a>,
 <a data-area="*q" data-idx="7" href="?where=nexearch&amp;qu

In [83]:
print("연관검색어")
[(tag.text, tag["href"]) for tag in dom.select("#nx_related_keywords ._related_keyword_ul li > a")]

연관검색어


[('미쓰에이 수지',
  '?where=nexearch&query=%EB%AF%B8%EC%93%B0%EC%97%90%EC%9D%B4+%EC%88%98%EC%A7%80&ie=utf8&sm=tab_she&qdt=0'),
 ('아이유',
  '?where=nexearch&query=%EC%95%84%EC%9D%B4%EC%9C%A0&ie=utf8&sm=tab_she&qdt=0'),
 ('원더랜드',
  '?where=nexearch&query=%EC%9B%90%EB%8D%94%EB%9E%9C%EB%93%9C&ie=utf8&sm=tab_she&qdt=0'),
 ('수지 나무위키',
  '?where=nexearch&query=%EC%88%98%EC%A7%80+%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4&ie=utf8&sm=tab_she&qdt=0'),
 ('수지 이민호',
  '?where=nexearch&query=%EC%88%98%EC%A7%80+%EC%9D%B4%EB%AF%BC%ED%98%B8&ie=utf8&sm=tab_she&qdt=0'),
 ('배가본드',
  '?where=nexearch&query=%EB%B0%B0%EA%B0%80%EB%B3%B8%EB%93%9C&ie=utf8&sm=tab_she&qdt=0'),
 ('이민호',
  '?where=nexearch&query=%EC%9D%B4%EB%AF%BC%ED%98%B8&ie=utf8&sm=tab_she&qdt=0'),
 ('수지 갤러리',
  '?where=nexearch&query=%EC%88%98%EC%A7%80+%EA%B0%A4%EB%9F%AC%EB%A6%AC&ie=utf8&sm=tab_she&qdt=0'),
 ('설현', '?where=nexearch&query=%EC%84%A4%ED%98%84&ie=utf8&sm=tab_she&qdt=0'),
 ('수지 이동욱',
  '?where=nexearch&query=%EC%88%98%EC%A7%80+%EC%9D%B4%EB%8F%99

# 다음 실습
    - CSS Selector만 이용

In [86]:
# 수지를 검색했을 때 나오는 쇼핑정보에서 제목, url, 가격만 가져오고 싶다!

url = "https://search.daum.net/search"
params = {'q':"수지"}

In [87]:
resp = download('get', url, params)

In [88]:
dom = BeautifulSoup(resp.text, 'lxml')

In [101]:
[(tag.strong.text, tag["href"], tag.find_all('div', {'class':'item_price'})[0].text) for tag in dom.select(".cont_shop > .list_shop.type_list .info_item ")]

[('20%쿠폰+복수구매할인! 데싱디바 수지 매니/페디 총집합',
  'http://shopping.daum.net/search/%EC%88%98%EC%A7%80/&docid:S3617010790&srchhow:Cexpo',
  ' 5,950원 '),
 ('[본사]데싱디바 수지 신상컬렉션 글로스 매니/페디 총집합',
  'http://shopping.daum.net/search/%EC%88%98%EC%A7%80/&docid:Y5293833921&srchhow:Cexpo',
  ' 5,950원 '),
 ('캉가루 톱코트 표준형 수지왁스 18.75L',
  'http://shopping.daum.net/siso/p/product/C3485387751/&q:%EC%88%98%EC%A7%80&cateid:104103107100&srchhow:Cexpo',
  ' 34,950원 최저 '),
 ('데싱디바 매직프레스 슈퍼 슬림핏 수지 빅스톤 매니 숏 사이즈',
  'http://shopping.daum.net/search/%EC%88%98%EC%A7%80/&docid:M6759159304&srchhow:Cexpo',
  ' 12,890원 '),
 ('데싱디바 매직프레스 슈퍼 슬림핏 수지네일 빅스톤 젤 페디',
  'http://shopping.daum.net/search/%EC%88%98%EC%A7%80/&docid:S6759346582&srchhow:Cexpo',
  ' 12,890원 '),
 ('[수지 귀걸이]시그니처D 드롭귀걸이 JDREPSF40ZC',
  'http://shopping.daum.net/search/%EC%88%98%EC%A7%80/&docid:J2857009405&srchhow:Cexpo',
  ' 159,880원 '),
 ('디디에두보(DIDIER DUBOT)][수지 귀걸이]라디디 귀걸이 JDDERSS01ZC',
  'http://shopping.daum.net/search/%EC%88%98%EC%A7%80/&docid:U3319402552&sr